# Audio Signal Classification using Long Short-Term Memory (LSTM) in TensorFlow

In this tutorial, we'll walk through the process of classifying audio signals using a Long Short-Term Memory (LSTM) neural network in TensorFlow. We'll cover data loading, preprocessing, model building, training, and evaluation.

We'll use the [UrbanSound8K](https://urbansounddataset.weebly.com/urbansound8k.html) dataset, which contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes.

## Step 1: Install Required Libraries

First, let's install the libraries we'll need. Make sure you have `tensorflow`, `librosa`, and `numpy` installed. You can install them using pip:

In [ ]:
!pip install tensorflow librosa numpy

## Step 2: Load and Preprocess the Data

We'll load the UrbanSound8K dataset and preprocess it. This involves reading the audio files, extracting features, and preparing the data for the LSTM model.

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load the UrbanSound8K dataset
DATASET_PATH = 'UrbanSound8K/audio/'
metadata = 'UrbanSound8K/metadata/UrbanSound8K.csv'

# Function to extract features from audio files
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None
    return mfccs_scaled

# Load metadata
import pandas as pd
metadata = pd.read_csv(metadata)

# Iterate through each sound file and extract features
features = []

for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(DATASET_PATH), 'fold'+str(row['fold'])+'/', str(row['slice_file_name']))
    class_label = row['class']
    data = extract_features(file_name)
    features.append([data, class_label])

# Convert into a Pandas DataFrame
features_df = pd.DataFrame(features, columns=['feature', 'class_label'])

# Convert features and corresponding classification labels into numpy arrays
X = np.array(features_df.feature.tolist())
y = np.array(features_df.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=42)

# Reshape for LSTM [samples, time steps, features]
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

## Step 3: Build the LSTM Model

Now, we'll build an LSTM model using TensorFlow. The model will have an LSTM layer followed by a Dense layer for classification.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Step 4: Train the Model

We'll train the model on our training data and validate it on the test data.

In [ ]:
history = model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), verbose=1)

## Step 5: Evaluate the Model

Finally, we'll evaluate the model's performance on the test data.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

## Conclusion

In this tutorial, we walked through the process of building and training a Long Short-Term Memory (LSTM) neural network for audio signal classification using TensorFlow. We used the UrbanSound8K dataset for our experiments and achieved a reasonable accuracy.